# Visualizations

The visualizations that power `interpret` use different renderers depending on the environment they are in. In most cases, the package will detect what kind of environment it is in and use the appropriate [renderer](https://github.com/interpretml/interpret/blob/683e632f4122af54eada2e214066de7be75bd7e0/python/interpret-core/interpret/provider/visualize.py#L25). There are times though when you want to forcefully select which one.

<h2>Dash Renderer</h2>

The Dash renderer is used for local environments such as running a Jupyter notebook on your laptop. It runs a Dash server, backed by Apache Flask in a separate process the first time its called by `interpret`.

This provides access to both embedded visualizations within notebooks as well as the full dashboard. However, due to requiring a live Flask server, it cannot render in an offline notebook.

See the source code for underestandings its configuration [here](https://github.com/interpretml/interpret/blob/683e632f4122af54eada2e214066de7be75bd7e0/python/interpret-core/interpret/provider/visualize.py#L149).

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import DashProvider
set_visualize_provider(DashProvider.from_address(('127.0.0.1', 7001)))

```{eval-rst}
.. autoclass:: interpret.provider.DashProvider
   :members:
   :inherited-members:
```

<h2>Inline Renderer</h2>

The inline renderer is used for cloud environments access to flask servers are not always available. In most configurations, it injects JavaScript in each notebook cell, including the bundle.

This does not allow for full dashboards, but it does allow offline support.

See the source code for underestandings its configuration [here](https://github.com/interpretml/interpret/blob/683e632f4122af54eada2e214066de7be75bd7e0/python/interpret-core/interpret/provider/visualize.py#L192).

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

```{eval-rst}
.. autoclass:: interpret.provider.InlineProvider
   :members:
   :inherited-members:
```

<h2>Interactivity</h2>

The visualizations consume the Interpret API, and is responsible
for both displaying explanations and the underlying rendering infrastructure.

<h2>Visualizing with the show method</h2>

Interpret exposes a top-level method `show`, of which acts as the surface for rendering explanation visualizations. This can produce either a drop down widget or dashboard depending on what's provided.

<h2>Show a single explanation</h2>

For basic use cases, it is good to show an explanation one at a time. The rendered widget will provide a dropdown to select between visualizations. For example, in the event of a global explanation, it will provide an overview, along with graphs for each feature as shown with the code below:

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
X = df.iloc[:, :-1]
y = (df.iloc[:, -1] == " >50K").astype(int)

seed = 42
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

In [ ]:
ebm_global = ebm.explain_global()
show(ebm_global)

<br/>
<br/>
<br/>
<br/>

<h2>Show a specific visualization within an explanation</h2>

Let's say you are after one specific visualization within an explanation, then you can specify it with a key as the subsequent function argument.

In [ ]:
show(ebm_global, "Age")

<br/>
<br/>
<br/>
<br/>

<h2>Show multiple explanations for comparison</h2>

If you running in a local environment (such as a running Python on your laptop), then `show` can expose a dashboard for comparison which can be invoked the in the following way (provide a list of explanations in the first argument):

In [ ]:
from interpret.glassbox import LogisticRegression

# We have to transform categorical variables to use Logistic Regression
X_train = pd.get_dummies(X_train, prefix_sep='.').astype(float)

lr = LogisticRegression(random_state=seed, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)

lr_global = lr.explain_global()

In [ ]:
show([ebm_global, lr_global])

<h2>Interpret API</h2>

The API is responsible for standardizing ML interpretability
explainers and explanations, providing a consistent interface for both users
and developers. To support this, it also provides foundational top-level methods that support
visualization and data access.

**Explainers** are glassbox or blackbox algorithms that will produce an **explanation**, an artifact that is ready for visualizations or further data processing.

<h2>Explainer</h2>

An explainer will produce an explanation from its `.explain_*` method. These explanations normally provide an understanding of **global** model behavior or **local** individual predictions (`.explain_global` and `.explain_local` respectively).

```{eval-rst}
.. autoclass:: interpret.api.base.ExplainerMixin
   :members:
   :inherited-members:
```

<h2>Explanation</h2>

An explanation is a self-contained object that help understands either its target model behavior, or a set of individual predictions. The explanation should provide access to visualizations through `.visualize`, and data processing the `.data` method. Both `.visualize` and `.data` should share the same function signature in terms of arguments.

```{eval-rst}
.. autoclass:: interpret.api.base.ExplanationMixin
   :members:
   :inherited-members:
```

<h2>Show</h2>

The `show` method is used as a universal function that provides visualizations for whatever explanation(s) is provided in its arguments. Implementation-wise it will provide some visualization platform (i.e. a dashboard or widget) and expose the explanation(s)' visualizations as given by the `.visualize` call.
